# Experimenting with several learners APIs for classification and regression

* ```MOAClassifier``` and ```MOARegressor```
* ```AdaptiveRandomForest```, ```OnlineBagging``` and ```AdaptiveRandomForestRegressor```
* ```ClassificationEvaluator```, ```ClassificationWindowedEvaluator```, ```RegressionEvaluator``` and ```RegressionWindowedEvaluator```

Some comments: 
* We can use the ```MOAClassifier``` and ```MOARegressor``` to execute any MOA classifier and regressor, respectively.
* We can create wrappers for each class, like the AdaptiveRandomForest wrapper. Using an alias in here as we don't want to conflict with the MOA AdaptiveRandomForest. 
```from ensembles import AdaptiveRandomForest as ARF```

**Notebook last update: 20/10/2023**

In [1]:
from prepare_jpype import start_jpype
import pandas as pd
start_jpype()

# MOA imports
from moa.core import Example, Utils
from moa.streams.generators import RandomTreeGenerator as MOA_RTG
from moa.streams import ArffFileStream
# For regression tests
from moa.classifiers.trees import FIMTDD
from moa.evaluation import BasicRegressionPerformanceEvaluator

# For classification tests
from moa.classifiers.trees import HoeffdingAdaptiveTree

# local code imports
from evaluation import *
from ensembles import OnlineBagging, AdaptiveRandomForest as ARF, AdaptiveRandomForestRegressor
from MOALearners import MOAClassifier, MOARegressor


arff_fried_path = './data/fried.arff'

from stream import *

MOA jar path location (config.ini): /Users/gomeshe/Dropbox/ciencia_computacao/dev/main-projects/MOABridge/jar/moa.jar
JVM Location (system): 
/Users/gomeshe/Library/Java/JavaVirtualMachines/openjdk-20.0.1/Contents/Home
JVM args: ['-Xmx8g', '-Xss10M']
Sucessfully started the JVM and added MOA jar to the class path


In [2]:
maxInstancesToProcess = 50000
sampleFrequency = 10000
instancesProcessed = 1

# stream = Stream(moa_stream=ArffFileStream(arff_fried_path, -1))
stream = ARFFStream(path=arff_fried_path)
learner = MOARegressor(schema=stream.get_schema(), moa_learner=FIMTDD())

evaluator_TTT = RegressionEvaluator(schema=stream.get_schema(), window_size=sampleFrequency)
evaluator_windowed = RegressionWindowedEvaluator(schema=stream.get_schema(), window_size=sampleFrequency)

while stream.has_more_instances() and instancesProcessed <= maxInstancesToProcess:
    instance = stream.next_instance()

    prediction = learner.predict(instance)

    # evaluator.addResult(instance, prediction)
    evaluator_TTT.update(instance.y(),prediction)
    evaluator_windowed.update(instance.y(),prediction)
    learner.train(instance)
    
    instancesProcessed += 1

print(evaluator_TTT.adjusted_R2())
evaluator_windowed.metrics_per_window()

0.6996453058512179


,classified instances,mean absolute error,root mean squared error,relative mean absolute error,relative root mean squared error,coefficient of determination,adjusted coefficient of determination
0,10000.0,2.845125,3.872532,0.702732,0.777990,0.394731,0.394064
1,10000.0,1.891743,2.423392,0.465342,0.484313,0.765441,0.765183
2,10000.0,1.744441,2.227354,0.432927,0.449547,0.797907,0.797685
3,10000.0,1.649495,2.106839,0.400250,0.416741,0.826327,0.826136


In [3]:
maxInstancesToProcess = 5000
sampleFrequency = 1000
instancesProcessed = 1

# stream = Stream(moa_stream=ArffFileStream(arff_fried_path, -1))
stream = RandomTreeGenerator()
# stream.get_schema().get_label_values() is None

learner = ARF(schema=stream.get_schema(), CLI="-s 5")

evaluator_TTT = ClassificationEvaluator(schema=stream.get_schema(), window_size=sampleFrequency)
evaluator_windowed = ClassificationWindowedEvaluator(schema=stream.get_schema(), window_size=sampleFrequency)

while stream.has_more_instances() and instancesProcessed <= maxInstancesToProcess:
    instance = stream.next_instance()

    prediction = learner.predict(instance)

    # evaluator.addResult(instance, prediction)
    evaluator_TTT.update(instance.y(),prediction)
    evaluator_windowed.update(instance.y(),prediction)
    learner.train(instance)
    
    instancesProcessed += 1

print(evaluator_TTT.accuracy())
evaluator_windowed.metrics_per_window()

77.24


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)
0,1000.0,70.5,32.261768,38.669439,21.122995
1,2000.0,75.8,47.834035,44.748858,40.099010
2,3000.0,78.5,54.505442,55.208333,47.303922
3,4000.0,79.8,57.780685,57.651992,51.789976
4,5000.0,81.6,60.958221,59.825328,54.114713


## Using the ```ClassificationEvaluator``` and ```ClassificationWindowedEvaluator``` with the ```MOA AdaptiveRandomForest```
* This example uses a MOA generator directly, the ```MOA_RTG()``` i.e. ```moa.streams.generators.RandomTreeGenerator```

In [4]:
from moa.classifiers.meta import AdaptiveRandomForest

maxInstancesToProcess = 1000
instancesProcessed = 1

learner = AdaptiveRandomForest()
# Setting parameters using setViaCLIString
learner.getOptions().setViaCLIString("-s 5")
learner.prepareForUse()

rtg = MOA_RTG()
# Setting parameters using setViaCLIString
rtg.getOptions().setViaCLIString("-c 3 -u 10 -o 0")
rtg.prepareForUse()

# Creating the schema without using the stream header (need to define the possible class labels)
#    and using the stream header (rtg.getHeader()) for illustration.
evaluator = ClassificationWindowedEvaluator(schema=Schema(labels=[0,1,2]), window_size=100, recall_per_class=True)
evaluator_TTT = ClassificationEvaluator(schema=Schema(moa_header=rtg.getHeader()), recall_per_class=True)

# learner.setModelContext(rtg.getHeader())

while rtg.hasMoreInstances() and instancesProcessed <= maxInstancesToProcess:
    instance = rtg.nextInstance()

    prediction = learner.getVotesForInstance(instance)

    evaluator.update(instance.getData().classValue(), Utils.maxIndex(prediction))
    evaluator_TTT.update(instance.getData().classValue(), Utils.maxIndex(prediction))
    learner.trainOnInstance(instance)

    instancesProcessed += 1

print(f"Test-Then-Train accuracy = {evaluator_TTT.accuracy()}")
evaluator.metrics_per_window()

Test-Then-Train accuracy = 82.0


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent)
0,100.0,75.0,51.399689,56.896552,34.210526,87.500000,70.270270,0.000000
1,200.0,77.0,56.340167,58.181818,52.083333,90.384615,76.923077,0.000000
2,300.0,79.0,61.671838,67.187500,54.347826,87.037037,88.571429,9.090909
3,400.0,85.0,73.661106,73.684211,71.698113,95.744681,88.095238,27.272727
4,500.0,83.0,70.164970,73.846154,64.583333,90.384615,83.783784,45.454545
5,600.0,89.0,81.443995,82.258065,78.000000,94.000000,91.666667,64.285714
6,700.0,82.0,69.141094,66.037736,66.666667,89.130435,90.243902,30.769231
7,800.0,83.0,72.039474,67.307692,64.583333,90.384615,96.428571,45.000000
8,900.0,84.0,72.096268,68.627451,65.957447,86.792453,94.444444,36.363636
9,1000.0,83.0,70.613656,67.924528,63.043478,90.740741,78.787879,61.538462


## Using two MOA Objects with the framework evaluators
* The ```MOAClassifier``` wrapper is used to wrap around the MOA learner ```HoeffdingAdaptiveTree()```
* The ```Instance``` wrapper is used to wrap around the MOA instance returned from ```rtg.nextInstance()```
* The evaluators are initialised with different ways of setting the ```Schema``` just to demonstrate flexibility
  * ```schema=Schema(labels=[0,1,2])```: when we know what are the class label values
  * ```schema=Schema(moa_header=rtg.getHeader())```: when we are using a moa generator

In [5]:
from stream import Instance

rtg = MOA_RTG()
# Setting parameters using setViaCLIString
rtg.getOptions().setViaCLIString("-c 3 -u 10 -o 0")
rtg.prepareForUse()

learner = MOAClassifier(random_seed=1, moa_learner=HoeffdingAdaptiveTree())

# Shows the CLI help, basically what each char in the CLI mean
print(learner.CLI_help())

evaluator = ClassificationWindowedEvaluator(schema=Schema(labels=[0,1,2]), window_size=100, recall_per_class=True)
evaluator_TTT = ClassificationEvaluator(schema=Schema(moa_header=rtg.getHeader()), recall_per_class=True)

maxInstancesToProcess = 1000
instancesProcessed = 1
while rtg.hasMoreInstances() and instancesProcessed <= maxInstancesToProcess:
    instance = Instance(rtg.nextInstance())
    prediction = learner.predict(instance)
    
    evaluator.update(instance.y(), prediction)
    evaluator_TTT.update(instance.y(), prediction)
    
    learner.train(instance)
    instancesProcessed += 1

print(f"Test-Then-Train accuracy = {evaluator_TTT.accuracy()}")
evaluator.metrics_per_window()

-m maxByteSize (default: 33554432)
Maximum memory consumed by the tree.
-n numericEstimator (default: GaussianNumericAttributeClassObserver)
Numeric estimator to use.
-d nominalEstimator (default: NominalAttributeClassObserver)
Nominal estimator to use.
-e memoryEstimatePeriod (default: 1000000)
How many instances between memory consumption checks.
-g gracePeriod (default: 200)
The number of instances a leaf should observe between split attempts.
-s splitCriterion (default: InfoGainSplitCriterion)
Split criterion to use.
-c splitConfidence (default: 1.0E-7)
The allowable error in split decision, values closer to 0 will take longer to decide.
-t tieThreshold (default: 0.05)
Threshold below which a split will be forced to break ties.
-b binarySplits
Only allow binary splits.
-z stopMemManagement
Stop growing as soon as memory limit is hit.
-r removePoorAtts
Disable poor attributes.
-p noPrePrune
Disable pre-pruning.
-l leafprediction (default: NBAdaptive)
Leaf prediction to use.
-q nbThr

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent)
0,100.0,75.0,52.033768,56.896552,34.210526,83.928571,70.270270,28.571429
1,200.0,75.0,53.789279,54.545455,47.916667,86.538462,74.358974,11.111111
2,300.0,66.0,40.382255,46.875000,26.086957,70.370370,77.142857,9.090909
3,400.0,76.0,57.431713,57.894737,54.716981,85.106383,83.333333,9.090909
4,500.0,71.0,49.766153,55.384615,39.583333,78.846154,78.378378,9.090909
5,600.0,78.0,62.749746,64.516129,56.000000,90.000000,80.555556,28.571429
6,700.0,77.0,60.413081,56.603774,57.407407,82.608696,85.365854,30.769231
7,800.0,76.0,59.404601,53.846154,50.000000,90.384615,67.857143,50.000000
8,900.0,75.0,56.001408,50.980392,46.808511,83.018868,80.555556,18.181818
9,1000.0,78.0,60.559340,58.490566,52.173913,90.740741,66.666667,53.846154


### Using the ```ClassificationEvaluator``` and ```ClassificationWindowedEvaluator``` with the wrapper for the AdaptiveRandomForest

In [6]:
rtg = MOA_RTG()
# Setting parameters using setViaCLIString
rtg.getOptions().setViaCLIString("-c 3 -u 10 -o 0")
rtg.prepareForUse()

schema = Schema(moa_header=rtg.getHeader())
learner = ARF(schema=schema, ensemble_size=5)

evaluator = ClassificationWindowedEvaluator(schema=Schema(labels=[0,1,2]), window_size=100, recall_per_class=True)
evaluator_TTT = ClassificationEvaluator(schema=Schema(moa_header=rtg.getHeader()), recall_per_class=True)

maxInstancesToProcess = 1000
instancesProcessed = 1
while rtg.hasMoreInstances() and instancesProcessed <= maxInstancesToProcess:
    instance = Instance(rtg.nextInstance())
    prediction = learner.predict(instance)
    
    evaluator.update(instance.y(), prediction)
    evaluator_TTT.update(instance.y(), prediction)
    
    learner.train(instance)
    instancesProcessed += 1

print(f"Test-Then-Train accuracy = {evaluator_TTT.accuracy()}")
evaluator.metrics_per_window()

Test-Then-Train accuracy = 82.0


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent)
0,100.0,75.0,51.399689,56.896552,34.210526,87.500000,70.270270,0.000000
1,200.0,77.0,56.340167,58.181818,52.083333,90.384615,76.923077,0.000000
2,300.0,79.0,61.671838,67.187500,54.347826,87.037037,88.571429,9.090909
3,400.0,85.0,73.661106,73.684211,71.698113,95.744681,88.095238,27.272727
4,500.0,83.0,70.164970,73.846154,64.583333,90.384615,83.783784,45.454545
5,600.0,89.0,81.443995,82.258065,78.000000,94.000000,91.666667,64.285714
6,700.0,82.0,69.141094,66.037736,66.666667,89.130435,90.243902,30.769231
7,800.0,83.0,72.039474,67.307692,64.583333,90.384615,96.428571,45.000000
8,900.0,84.0,72.096268,68.627451,65.957447,86.792453,94.444444,36.363636
9,1000.0,83.0,70.613656,67.924528,63.043478,90.740741,78.787879,61.538462


### Using the ```ClassificationEvaluator``` and ```ClassificationWindowedEvaluator``` with the wrapper for the OnlineBagging

In [7]:
rtg = MOA_RTG()
# Setting parameters using setViaCLIString
rtg.getOptions().setViaCLIString("-c 3 -u 10 -o 0")
rtg.prepareForUse()

schema = Schema(moa_header=rtg.getHeader())
learner = OnlineBagging(schema=schema, ensemble_size=10, base_learner="(trees.HoeffdingTree -g 50)")
# learner = OnlineBagging(schema=schema, CLI="-s 10")

evaluator = ClassificationWindowedEvaluator(schema=Schema(labels=[0,1,2]), window_size=100, recall_per_class=True)
evaluator_TTT = ClassificationEvaluator(schema=Schema(moa_header=rtg.getHeader()), recall_per_class=True)

maxInstancesToProcess = 1000
instancesProcessed = 1
while rtg.hasMoreInstances() and instancesProcessed <= maxInstancesToProcess:
    instance = Instance(rtg.nextInstance())
    prediction = learner.predict(instance)
    
    evaluator.update(instance.y(), prediction)
    evaluator_TTT.update(instance.y(), prediction)
    
    learner.train(instance)
    instancesProcessed += 1

print(f"Test-Then-Train accuracy = {evaluator_TTT.accuracy()}")
evaluator.metrics_per_window()

Test-Then-Train accuracy = 75.6


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent)
0,100.0,74.0,49.455677,55.172414,31.578947,85.714286,67.567568,14.285714
1,200.0,77.0,57.177434,58.181818,52.083333,90.384615,74.358974,11.111111
2,300.0,73.0,50.675923,57.812500,41.304348,81.481481,82.857143,0.000000
3,400.0,72.0,50.062422,50.877193,47.169811,78.723404,80.952381,9.090909
4,500.0,73.0,51.015965,58.461538,43.750000,82.692308,78.378378,9.090909
5,600.0,78.0,60.700250,64.516129,56.000000,92.000000,83.333333,14.285714
6,700.0,75.0,56.070989,52.830189,53.703704,86.956522,82.926829,7.692308
7,800.0,73.0,52.195467,48.076923,43.750000,94.230769,71.428571,20.000000
8,900.0,81.0,65.697779,62.745098,59.574468,90.566038,80.555556,36.363636
9,1000.0,80.0,63.728691,62.264151,56.521739,96.296296,60.606061,61.538462


### Using the ```ClassificationEvaluator``` and ```ClassificationWindowedEvaluator``` with the wrapper for the AdaptiveRandomForest

* This experiment shows how we can use the CLI to setup the object directly instead of setting each parameter individually. 

In [8]:
rtg = MOA_RTG()
# Setting parameters using setViaCLIString
rtg.getOptions().setViaCLIString("-c 3 -u 10 -o 0")
rtg.prepareForUse()

learner = ARF(random_seed=1, CLI="-s 5")

# Shows the CLI help, basically what each char in the CLI mean
# print(learner.CLI_help())

evaluator = ClassificationWindowedEvaluator(schema=Schema(labels=[0,1,2]), window_size=100, recall_per_class=True)
evaluator_TTT = ClassificationEvaluator(schema=Schema(moa_header=rtg.getHeader()), recall_per_class=True)

maxInstancesToProcess = 1000
instancesProcessed = 1
while rtg.hasMoreInstances() and instancesProcessed <= maxInstancesToProcess:
    instance = Instance(rtg.nextInstance())
    prediction = learner.predict(instance)
    
    evaluator.update(instance.y(), prediction)
    evaluator_TTT.update(instance.y(), prediction)
    
    learner.train(instance)
    instancesProcessed += 1

print(f"Test-Then-Train accuracy = {evaluator_TTT.accuracy()}")
evaluator.metrics_per_window()

Test-Then-Train accuracy = 82.0


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent)
0,100.0,75.0,51.399689,56.896552,34.210526,87.500000,70.270270,0.000000
1,200.0,77.0,56.340167,58.181818,52.083333,90.384615,76.923077,0.000000
2,300.0,79.0,61.671838,67.187500,54.347826,87.037037,88.571429,9.090909
3,400.0,85.0,73.661106,73.684211,71.698113,95.744681,88.095238,27.272727
4,500.0,83.0,70.164970,73.846154,64.583333,90.384615,83.783784,45.454545
5,600.0,89.0,81.443995,82.258065,78.000000,94.000000,91.666667,64.285714
6,700.0,82.0,69.141094,66.037736,66.666667,89.130435,90.243902,30.769231
7,800.0,83.0,72.039474,67.307692,64.583333,90.384615,96.428571,45.000000
8,900.0,84.0,72.096268,68.627451,65.957447,86.792453,94.444444,36.363636
9,1000.0,83.0,70.613656,67.924528,63.043478,90.740741,78.787879,61.538462


### MOARegressor tests

### Using the MOA regressor FIMTDD directly (without wrapping it with ```MOARegressor```)

In [9]:
maxInstancesToProcess = 50000
sampleFrequency = 10000
instancesProcessed = 1

fried_arff = ArffFileStream(arff_fried_path, -1)
fried_arff.prepareForUse()

learner = FIMTDD()
learner.prepareForUse()

# Setting parameters using the option attribute directly
evaluator = BasicRegressionPerformanceEvaluator()
learner.setModelContext(fried_arff.getHeader())

# Create empty lists to store the data
data = []
performance_names = []
performance_values = []

while fried_arff.hasMoreInstances() and instancesProcessed <= maxInstancesToProcess:
    instance = fried_arff.nextInstance()

    prediction = learner.getVotesForInstance(instance)

    evaluator.addResult(instance, prediction)
    learner.trainOnInstance(instance)

    if instancesProcessed == 1:
        performance_measurements = evaluator.getPerformanceMeasurements()
        performance_names = ["".join(measurement.getName()) for measurement in performance_measurements]

    if instancesProcessed % sampleFrequency == 0:
        performance_values = [measurement.getValue() for measurement in evaluator.getPerformanceMeasurements()]
        data.append(performance_values)
    
    instancesProcessed += 1

# Create a DataFrame using collected data
results_df = pd.DataFrame(data, columns=performance_names)

# Print the DataFrame
results_df

,classified instances,mean absolute error,root mean squared error,relative mean absolute error,relative root mean squared error,coefficient of determination,adjusted coefficient of determination
0,10000.0,2.845125,3.872532,0.702842,0.778102,0.394557,0.393890
1,20000.0,2.368434,3.230273,0.583849,0.647311,0.580988,0.580758
2,30000.0,2.160436,2.934305,0.533814,0.589412,0.652593,0.652466
3,40000.0,2.032701,2.750873,0.499972,0.550427,0.697030,0.696947


## Testing the ```MOARegressor``` wrapper and the ```BasicRegressionPerformanceEvaluator``` from MOA directly. 

In [10]:
from evaluation import RegressionEvaluator
from stream import ARFFStream

maxInstancesToProcess = 50000
sampleFrequency = 10000
instanceProcessed = 1

fried_stream = ARFFStream(path=arff_fried_path, class_index=-1)

learner = MOARegressor(schema=fried_stream.get_schema(), moa_learner=FIMTDD())
evaluator = RegressionEvaluator(schema=fried_stream.get_schema(), window_size=sampleFrequency)

while fried_stream.has_more_instances() and instancesProcessed <= maxInstancesToProcess:
    instance = fried_stream.next_instance()

    prediction = learner.predict(instance)

    evaluator.update(instance.y(), prediction)
    learner.train(instance)

    instanceProcessed += 1

print(evaluator)

{'classified instances': 40768.0, 'mean absolute error': 2.024111913389299, 'root mean squared error': 2.739036408897116, 'relative mean absolute error': 0.4978639452538999, 'relative root mean squared error': 0.5479723082580009, 'coefficient of determination': 0.6997263493823984, 'adjusted coefficient of determination': 0.6996453058512179}


## Using the ```MOARegressor``` to wrap the FIMTDD learner and evaluating it using ```RegressionEvaluator``` and ```RegressionWindowedEvaluator```

In [11]:
from stream import stream_from_file

maxInstancesToProcess = 50000
sampleFrequency = 10000
instancesProcessed = 1

fried_arff = stream_from_file(path_to_csv_or_arff=arff_fried_path, class_index=-1)

learner = MOARegressor(schema=fried_arff.get_schema(), moa_learner=FIMTDD())

evaluator_TTT = RegressionEvaluator(schema=fried_arff.get_schema(), window_size=sampleFrequency)
evaluator_windowed = RegressionWindowedEvaluator(schema=fried_arff.get_schema(), window_size=sampleFrequency)

while fried_arff.has_more_instances() and instancesProcessed <= maxInstancesToProcess:
    instance = fried_arff.next_instance()

    prediction = learner.predict(instance)

    evaluator_TTT.update(instance.y(),prediction)
    evaluator_windowed.update(instance.y(),prediction)
    learner.train(instance)
    
    instancesProcessed += 1

print(evaluator_TTT.MAE())
print(evaluator_TTT.RMSE())
print(evaluator_TTT.RMAE())
print(evaluator_TTT.R2())
print(evaluator_TTT.adjusted_R2())
evaluator_windowed.metrics_per_window()

2.024111913389299
2.739036408897116
0.4978639452538999
0.6997263493823984
0.6996453058512179


,classified instances,mean absolute error,root mean squared error,relative mean absolute error,relative root mean squared error,coefficient of determination,adjusted coefficient of determination
0,10000.0,2.845125,3.872532,0.702732,0.777990,0.394731,0.394064
1,10000.0,1.891743,2.423392,0.465342,0.484313,0.765441,0.765183
2,10000.0,1.744441,2.227354,0.432927,0.449547,0.797907,0.797685
3,10000.0,1.649495,2.106839,0.400250,0.416741,0.826327,0.826136


## Testing the ```AdaptiveRandomForestRegressor``` wrapper

In [12]:
maxInstancesToProcess = 50000
instancesProcessed = 1
sampleFrequency = 10000

fried_arff = stream_from_file(path_to_csv_or_arff=arff_fried_path, class_index=-1)

learner = AdaptiveRandomForestRegressor(schema=fried_arff.get_schema(), ensemble_size=5)

evaluator_TTT = RegressionEvaluator(schema=fried_arff.get_schema(), window_size=sampleFrequency)
evaluator_windowed = RegressionWindowedEvaluator(schema=fried_arff.get_schema(), window_size=sampleFrequency)

while fried_arff.has_more_instances() and instancesProcessed <= maxInstancesToProcess:
    instance = fried_arff.next_instance()

    prediction = learner.predict(instance)
    
    evaluator_TTT.update(instance.y(),prediction)
    evaluator_windowed.update(instance.y(),prediction)
    learner.train(instance)
    
    instancesProcessed += 1

print(evaluator_TTT.MAE())
print(evaluator_TTT.RMSE())
print(evaluator_TTT.RMAE())
print(evaluator_TTT.R2())
print(evaluator_TTT.adjusted_R2())
evaluator_windowed.metrics_per_window()

2.87204803582077
3.771523667554421
0.7064279186411917
0.4306815105163202
0.4305278520762299


,classified instances,mean absolute error,root mean squared error,relative mean absolute error,relative root mean squared error,coefficient of determination,adjusted coefficient of determination
0,10000.0,3.171940,4.105194,0.783453,0.824732,0.319817,0.319068
1,10000.0,2.842502,3.731284,0.699215,0.745694,0.443941,0.443329
2,10000.0,2.781082,3.654003,0.690196,0.737488,0.456112,0.455513
3,10000.0,2.711143,3.594034,0.657858,0.710913,0.494602,0.494046
